In [1]:
import json
from pathlib import Path

import geopandas as gpd
from sqlalchemy import create_engine, text as sql_text

from pyagnps import soil_data_market as sdm

In [2]:
import importlib

importlib.reload(sdm);

In [3]:
credentials = Path('../../inputs/db_credentials.json')
with open(credentials, 'r') as f:
    credentials = json.load(f)

user     = credentials['user']
password = credentials['password']
host     = credentials['host']
port     = credentials['port']
database = credentials['database']

In [5]:
# path_to_storage_geo_ssurgo = Path('D:/AIMS/Datasets/Soil/SSURGO/thuc_ssurgo_clips/')
path_to_storage_geo_ssurgo = Path('../../outputs/shapefiles/thuc_ssurgo_clips/')
path_to_storage_geo_ssurgo.mkdir(parents=True, exist_ok=True)

Query the cells to attribute soil data to

In [6]:
thuc_id = '1002'

In [7]:
SRID = 4326
# lat, lon = 38.0446312, -87.8245648
lat, lon = 38.9327257,-87.8815463

# query = f'SELECT * FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;

query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_ids"

In [8]:
# create a SQLAlchemy engine object
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

In [9]:
with engine.connect() as conn:
    cells = gpd.read_postgis(sql=sql_text(query), con=conn, geom_col='geom')

In [10]:
# path_to_thucs = Path('../../inputs/thucs/tophuc_S_M_40000_closed_holes_with_container_thuc_merged_bbox_area_first_kept.gpkg')
# thucs = gpd.read_file(path_to_thucs)

# path_to_cells = Path('../../outputs/topagnps/johnson_long_creek/AnnAGNPS_Cell_IDs.gpkg')

# cells = gpd.read_file(path_to_cells)
# cells = cells.rename(columns={'DN': 'dn', 'geometry': 'geom'})
# cells = cells.set_geometry('geom')

In [11]:
utm = cells.estimate_utm_crs()

Download SSURGO soil data layer for the area

In [12]:
bbox = cells.envelope.buffer(60).to_crs(epsg=4326).total_bounds

In [14]:
geo_soil = sdm.download_soil_geodataframe_tiles(bbox, tile_size=0.1)

  2%|▏         | 3/150 [00:31<25:33, 10.43s/it]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
geo_soil = geo_soil.to_crs(utm)
geo_soil = geo_soil.to_file(path_to_storage_geo_ssurgo / f'thuc_{thuc_id}_geo_ssurgo.gpkg', driver='GPKG')

In [11]:
geo_soil = gpd.read_file(path_to_storage_geo_ssurgo / f'thuc_{thuc_id}_geo_ssurgo.gpkg')

Check that the soil covers the cells

In [12]:
# Convert soil and cells to same CRS
cells = cells.to_crs(utm)

In [28]:
if not(cells.within(geo_soil.unary_union.envelope).all()):
    print('Not all cells are covered by the soil geometry file')
else:
    print('OK to proceed')

Apply `Soil_ID` to each cell

In [13]:
cells = sdm.assign_attr_plurality_vector_layer(cells, geo_soil, attr='mukey', bin_id='dn')

In [14]:
cells = cells.rename(columns={'dn': 'cell_id', 'mukey': 'soil_id'})
cells.sort_values(by='cell_id')

,fid,cell_id,geom,soil_id
584077,584078,22,"POLYGON ((696888.788 3577880.539, 696888.788 3...",808562
584078,584079,23,"POLYGON ((697698.788 3577700.539, 697698.788 3...",568708
584076,584077,31,"POLYGON ((696888.788 3578240.539, 696888.788 3...",808541
584074,584075,32,"POLYGON ((696918.788 3577880.539, 696918.788 3...",808562
584071,584072,32,"POLYGON ((696888.788 3577910.539, 696888.788 3...",808562
...,...,...,...,...
584033,584034,1395753,"POLYGON ((698028.788 3578810.539, 698028.788 3...",633319
584060,584061,1395753,"POLYGON ((697758.788 3578540.539, 697758.788 3...",633319
584050,584051,1395753,"POLYGON ((697908.788 3578690.539, 697908.788 3...",633319
584065,584066,1395761,"POLYGON ((698028.788 3578540.539, 698028.788 3...",2693455


In [31]:
# cells.explore(column='soil_id')

In [15]:
data_to_update = cells[['cell_id', 'soil_id']].to_dict(orient='records')

Attribute `soil_id` to every `cell_id` in the database

In [17]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

# create a SQLAlchemy engine object
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# create a session factory
Session = sessionmaker(bind=engine)

# create a new session
session = Session()

# create a transaction
transaction = session.begin()

try:
    # execute your update query here
    query = f"UPDATE thuc_{thuc_id}_annagnps_cell_data_section SET soil_id = :soil_id WHERE cell_id = :cell_id"

    session.execute(sql_text(query), data_to_update)

    # commit the transaction
    transaction.commit()

except SQLAlchemyError as e:
    # rollback the transaction on error
    transaction.rollback()
    print(e)

finally:
    # close the session
    session.close()